In [14]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [15]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-11 07:35:07--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-05-11 07:35:09 (1.22 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Project_Database").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [17]:
from pyspark import SparkFiles
url = "https://kwasib-bucket.s3.amazonaws.com/openpowerlifting.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("openpowerlifting.csv"), header=True, inferSchema=True)

In [18]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()

+--------------------+---+-----+---------+----+--------+--------+------------+-------------+--------+--------+--------+--------+------------+--------+--------+--------+--------+------------+-----------+-----------+-----------+-----------+---------------+-------+-----+------+---------+------------+---------+------+---------+----------+----------+-----------+---------+-------------+
|                Name|Sex|Event|Equipment| Age|AgeClass|Division|BodyweightKg|WeightClassKg|Squat1Kg|Squat2Kg|Squat3Kg|Squat4Kg|Best3SquatKg|Bench1Kg|Bench2Kg|Bench3Kg|Bench4Kg|Best3BenchKg|Deadlift1Kg|Deadlift2Kg|Deadlift3Kg|Deadlift4Kg|Best3DeadliftKg|TotalKg|Place| Wilks|McCulloch|Glossbrenner|IPFPoints|Tested|  Country|Federation|      Date|MeetCountry|MeetState|     MeetName|
+--------------------+---+-----+---------+----+--------+--------+------------+-------------+--------+--------+--------+--------+------------+--------+--------+--------+--------+------------+-----------+-----------+-----------+------

In [19]:
df.columns

['Name',
 'Sex',
 'Event',
 'Equipment',
 'Age',
 'AgeClass',
 'Division',
 'BodyweightKg',
 'WeightClassKg',
 'Squat1Kg',
 'Squat2Kg',
 'Squat3Kg',
 'Squat4Kg',
 'Best3SquatKg',
 'Bench1Kg',
 'Bench2Kg',
 'Bench3Kg',
 'Bench4Kg',
 'Best3BenchKg',
 'Deadlift1Kg',
 'Deadlift2Kg',
 'Deadlift3Kg',
 'Deadlift4Kg',
 'Best3DeadliftKg',
 'TotalKg',
 'Place',
 'Wilks',
 'McCulloch',
 'Glossbrenner',
 'IPFPoints',
 'Tested',
 'Country',
 'Federation',
 'Date',
 'MeetCountry',
 'MeetState',
 'MeetName']

In [20]:
df= df.drop('Federation','Tested','IPFPoints','Glossbrenner','McCulloch','Equipment','Deadlift4Kg','Bench4Kg','Squat4Kg','Division','Country')
df.show()

+--------------------+---+-----+----+--------+------------+-------------+--------+--------+--------+------------+--------+--------+--------+------------+-----------+-----------+-----------+---------------+-------+-----+------+----------+-----------+---------+-------------+
|                Name|Sex|Event| Age|AgeClass|BodyweightKg|WeightClassKg|Squat1Kg|Squat2Kg|Squat3Kg|Best3SquatKg|Bench1Kg|Bench2Kg|Bench3Kg|Best3BenchKg|Deadlift1Kg|Deadlift2Kg|Deadlift3Kg|Best3DeadliftKg|TotalKg|Place| Wilks|      Date|MeetCountry|MeetState|     MeetName|
+--------------------+---+-----+----+--------+------------+-------------+--------+--------+--------+------------+--------+--------+--------+------------+-----------+-----------+-----------+---------------+-------+-----+------+----------+-----------+---------+-------------+
|        Abbie Murphy|  F|  SBD|29.0|   24-34|        59.8|           60|    80.0|    92.5|   105.0|       105.0|    45.0|    50.0|    55.0|        55.0|      110.0|      120.0| 

In [21]:
df= df.dropna()
df.show()

+--------------------+---+-----+----+--------+------------+-------------+--------+--------+--------+------------+--------+--------+--------+------------+-----------+-----------+-----------+---------------+-------+-----+------+----------+-----------+---------+-------------+
|                Name|Sex|Event| Age|AgeClass|BodyweightKg|WeightClassKg|Squat1Kg|Squat2Kg|Squat3Kg|Best3SquatKg|Bench1Kg|Bench2Kg|Bench3Kg|Best3BenchKg|Deadlift1Kg|Deadlift2Kg|Deadlift3Kg|Best3DeadliftKg|TotalKg|Place| Wilks|      Date|MeetCountry|MeetState|     MeetName|
+--------------------+---+-----+----+--------+------------+-------------+--------+--------+--------+------------+--------+--------+--------+------------+-----------+-----------+-----------+---------------+-------+-----+------+----------+-----------+---------+-------------+
|        Abbie Murphy|  F|  SBD|29.0|   24-34|        59.8|           60|    80.0|    92.5|   105.0|       105.0|    45.0|    50.0|    55.0|        55.0|      110.0|      120.0| 

In [22]:
first_place_df= df[df['Place'] == 1]
first_place_df.show()

+----------------+---+-----+----+--------+------------+-------------+--------+--------+--------+------------+--------+--------+--------+------------+-----------+-----------+-----------+---------------+-------+-----+------+----------+-----------+---------+-------------------+
|            Name|Sex|Event| Age|AgeClass|BodyweightKg|WeightClassKg|Squat1Kg|Squat2Kg|Squat3Kg|Best3SquatKg|Bench1Kg|Bench2Kg|Bench3Kg|Best3BenchKg|Deadlift1Kg|Deadlift2Kg|Deadlift3Kg|Best3DeadliftKg|TotalKg|Place| Wilks|      Date|MeetCountry|MeetState|           MeetName|
+----------------+---+-----+----+--------+------------+-------------+--------+--------+--------+------------+--------+--------+--------+------------+-----------+-----------+-----------+---------------+-------+-----+------+----------+-----------+---------+-------------------+
|      Ash Morgan|  F|  SBD|23.0|   20-23|        59.8|           60|   115.0|   125.0|  -135.0|       125.0|    65.0|    70.0|   -75.0|        70.0|      130.0|      140.0

In [23]:
# Create Competitor Performance table Dataframe
competitor_perf_df= first_place_df.select(["Name", "Best3SquatKg","Best3BenchKg", "Best3DeadliftKg", "TotalKg"])
competitor_perf_df.show()

+----------------+------------+------------+---------------+-------+
|            Name|Best3SquatKg|Best3BenchKg|Best3DeadliftKg|TotalKg|
+----------------+------------+------------+---------------+-------+
|      Ash Morgan|       125.0|        70.0|          150.0|  345.0|
|Brooke Kowalczyk|       200.0|        95.0|          180.0|  475.0|
|Dakoda Plumridge|       182.5|       105.0|          205.0|  492.5|
|   Helene Faccio|       137.5|        70.0|          182.5|  390.0|
|   Rachel Flavin|       150.0|        70.0|          122.5|  342.5|
|     Sarah Green|       210.0|        90.0|          185.0|  485.0|
|   Yasmin Moyses|       120.0|        77.5|          125.0|  322.5|
|      Chris Lepp|       272.5|       160.0|          267.5|  700.0|
|   Chris Tyrrell|       345.0|       190.0|          285.0|  820.0|
|      Emad Nayef|       272.5|       162.5|          280.0|  715.0|
|  Griffin Spence|       182.5|       107.5|          245.0|  535.0|
|       Joe Ordon|       290.0|   

In [24]:
# Create competitor demographics table
competitor_demo_df= first_place_df.select(["Name", "Sex", "Age","AgeClass","BodyweightKg", "WeightClassKg"])
competitor_demo_df.show()

+----------------+---+----+--------+------------+-------------+
|            Name|Sex| Age|AgeClass|BodyweightKg|WeightClassKg|
+----------------+---+----+--------+------------+-------------+
|      Ash Morgan|  F|23.0|   20-23|        59.8|           60|
|Brooke Kowalczyk|  F|37.0|   35-39|        74.8|           75|
|Dakoda Plumridge|  F|27.0|   24-34|        78.6|         82.5|
|   Helene Faccio|  F|50.0|   50-54|        55.2|           56|
|   Rachel Flavin|  F|35.0|   35-39|        88.9|           90|
|     Sarah Green|  F|35.0|   35-39|       116.7|         110+|
|   Yasmin Moyses|  F|31.0|   24-34|        51.3|           52|
|      Chris Lepp|  M|26.0|   24-34|        82.5|         82.5|
|   Chris Tyrrell|  M|29.0|   24-34|       124.8|          125|
|      Emad Nayef|  M|41.0|   40-44|        88.5|           90|
|  Griffin Spence|  M|27.0|   24-34|        67.1|         67.5|
|       Joe Ordon|  M|33.0|   24-34|       162.0|         140+|
|   Luke Faulkner|  M|32.0|   24-34|    

In [27]:
# Create winners table
winners_df = first_place_df.select(["Name", "Event", "Date", "MeetName","Place"])
winners_df.show()


+----------------+-----+----------+-------------------+-----+
|            Name|Event|      Date|           MeetName|Place|
+----------------+-----+----------+-------------------+-----+
|      Ash Morgan|  SBD|2018-10-27|      Melbourne Cup|    1|
|Brooke Kowalczyk|  SBD|2018-10-27|      Melbourne Cup|    1|
|Dakoda Plumridge|  SBD|2018-10-27|      Melbourne Cup|    1|
|   Helene Faccio|  SBD|2018-10-27|      Melbourne Cup|    1|
|   Rachel Flavin|  SBD|2018-10-27|      Melbourne Cup|    1|
|     Sarah Green|  SBD|2018-10-27|      Melbourne Cup|    1|
|   Yasmin Moyses|  SBD|2018-10-27|      Melbourne Cup|    1|
|      Chris Lepp|  SBD|2018-10-27|      Melbourne Cup|    1|
|   Chris Tyrrell|  SBD|2018-10-27|      Melbourne Cup|    1|
|      Emad Nayef|  SBD|2018-10-27|      Melbourne Cup|    1|
|  Griffin Spence|  SBD|2018-10-27|      Melbourne Cup|    1|
|       Joe Ordon|  SBD|2018-10-27|      Melbourne Cup|    1|
|   Luke Faulkner|  SBD|2018-10-27|      Melbourne Cup|    1|
|  Warri

In [29]:
# Create wilks score table
winners_wilk_score_df = first_place_df.select(["Name","Sex","BodyweightKg", "TotalKg", "Wilks"])
winners_wilk_score_df.show()

+----------------+---+------------+-------+------+
|            Name|Sex|BodyweightKg|TotalKg| Wilks|
+----------------+---+------------+-------+------+
|      Ash Morgan|  F|        59.8|  345.0|385.63|
|Brooke Kowalczyk|  F|        74.8|  475.0|452.31|
|Dakoda Plumridge|  F|        78.6|  492.5|455.17|
|   Helene Faccio|  F|        55.2|  390.0|464.08|
|   Rachel Flavin|  F|        88.9|  342.5|297.48|
|     Sarah Green|  F|       116.7|  485.0|389.79|
|   Yasmin Moyses|  F|        51.3|  322.5|406.26|
|      Chris Lepp|  M|        82.5|  700.0|468.93|
|   Chris Tyrrell|  M|       124.8|  820.0|467.43|
|      Emad Nayef|  M|        88.5|  715.0|460.44|
|  Griffin Spence|  M|        67.1|  535.0|414.47|
|       Joe Ordon|  M|       162.0|  765.0|418.61|
|   Luke Faulkner|  M|        98.8|  755.0|461.74|
|  Warrick Eccles|  M|       137.4|  900.0|504.36|
|    Ace Kirkwood|  M|        74.5|  612.5|438.49|
|  Billa Hamilton|  F|       67.05|  460.0|471.75|
|     Fiona Kelly|  F|       90